In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!git clone https://github.com/Sourajit-Maity/juvdv2-vdvwc.git

In [ ]:
!pip install ultralytics

In [ ]:
import shutil

# Define the source and destination paths
source_folder = '/kaggle/working/juvdv2-vdvwc/Val'
destination_folder = '/kaggle/working/juvdv2-vdvwc/images/'

# Copy the entire folder
shutil.move(source_folder, destination_folder)


In [ ]:
!pwd


In [ ]:
cd "/kaggle/working/juvdv2-vdvwc"

In [ ]:
mkdir images labels

In [ ]:
import os

# Define the current and new folder names
current_folder = '/kaggle/working/juvdv2-vdvwc/Annotation'
new_folder = '/kaggle/working/juvdv2-vdvwc/labels'

# Rename the folder
os.rename(current_folder, new_folder)


In [ ]:
import os
import cv2
from ultralytics import YOLO
from PIL import Image, ImageEnhance

# Define weather-specific preprocessing functions
def preprocess_image(image_path, label):
    image = cv2.imread(image_path)
    if 'Rainny' in label:
        image = denoise_image(image)
    if 'Night' in label:
        image = enhance_image(image)
    return image

def enhance_image(image):
    pil_image = Image.fromarray(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    enhancer = ImageEnhance.Brightness(pil_image)
    enhanced_image = enhancer.enhance(1.5)  # Increase brightness
    return cv2.cvtColor(np.array(enhanced_image), cv2.COLOR_RGB2BGR)

def denoise_image(image):
    denoised_image = cv2.fastNlMeansDenoisingColored(image, None, 10, 10, 7, 21)
    return denoised_image

def preprocess_annotations(annotation_path, image_shape):
    with open(annotation_path, 'r') as file:
        annotations = file.readlines()
    
    processed_annotations = []
    for annotation in annotations:
        parts = annotation.strip().split()
        try:
            class_index = int(parts[0])  # YOLO class index
            x_center = float(parts[1])
            y_center = float(parts[2])
            width = float(parts[3])
            height = float(parts[4])
        except ValueError as e:
            print(f"Error parsing annotation {annotation} in file {annotation_path}: {e}")
            continue
        
        # Convert YOLO format to box coordinates (x_min, y_min, x_max, y_max)
        x_min = int((x_center - width / 2) * image_shape[1])
        y_min = int((y_center - height / 2) * image_shape[0])
        x_max = int((x_center + width / 2) * image_shape[1])
        y_max = int((y_center + height / 2) * image_shape[0])
        
        processed_annotations.append([x_min, y_min, x_max, y_max, class_index])
    
    return processed_annotations

# Load image paths
def load_image_paths(root_folder):
    image_paths = []
    for folder in ['Train', 'Val']:
        for weather in ['Rainny', 'Sunny']:
            for time_of_day in ['Day', 'Night']:
                image_folder = os.path.join(root_folder, 'images', folder, weather, time_of_day)
                for file in os.listdir(image_folder):
                    if file.endswith('.jpg') or file.endswith('.png'):
                        image_paths.append(os.path.join(image_folder, file))
    return image_paths

# Load annotation paths
def load_annotation_paths(root_folder):
    annotation_paths = []
    for folder in ['Train', 'Val']:
        for weather in ['Rainny', 'Sunny']:
            for time_of_day in ['Day', 'Night']:
                annotation_folder = os.path.join(root_folder, 'labels', folder, weather, time_of_day)
                for file in os.listdir(annotation_folder):
                    if file.endswith('.txt') and file != 'train.txt':
                        annotation_paths.append(os.path.join(annotation_folder, file))
    return annotation_paths

# Preprocess and save images and annotations
def preprocess_and_save(root_folder):
    image_paths = load_image_paths(root_folder)
    annotation_paths = load_annotation_paths(root_folder)

    for img_path, annot_path in zip(image_paths, annotation_paths):
        label = os.path.basename(os.path.dirname(os.path.dirname(img_path)))  # Extract label from folder name
        image = preprocess_image(img_path, label)
        annotations = preprocess_annotations(annot_path, image.shape)

        # Save preprocessed images and annotations
        cv2.imwrite(img_path, image)
        with open(annot_path, 'w') as file:
            for annot in annotations:
                file.write(f"{annot[4]} {annot[0]} {annot[1]} {annot[2]} {annot[3]}\n")
                
def compute_metrics(predictions, ground_truths):
    # Convert predictions and ground truths to the format expected by sklearn
    y_true, y_pred = [], []
    for gt, pred in zip(ground_truths, predictions):
        y_true.append(gt[4])
        y_pred.append(pred[4])

    precision, recall, f1, _ = precision_recall_fscore_support(y_true, y_pred, average='binary')
    ap = average_precision_score(y_true, y_pred)
    return precision, recall, f1, ap

def evaluate_model(model, dataloader):
    predictions, ground_truths = [], []
    for batch in dataloader:
        images, targets = batch
        outputs = model(images)
        predictions.extend(outputs)
        ground_truths.extend(targets)

    precision, recall, f1, ap = compute_metrics(predictions, ground_truths)
    print(f"Precision: {precision:.4f}, Recall: {recall:.4f}, F1 Score: {f1:.4f}, Average Precision (AP): {ap:.4f}")

# YOLOv8s model training and evaluation
def main():
    # Set paths and parameters
    root_folder = '/kaggle/working/juvdv2-vdvwc'  # Update with your dataset root folder path
    batch_size = 16
    epochs = 2
    input_shape = (416, 416)  # Assuming input image size for YOLOv8

    # Preprocess and save data
    preprocess_and_save(root_folder)

    # Initialize YOLOv8 model from Ultralytics
    model = YOLO('yolov8s.pt')  # Load the pre-trained YOLOv8s model

    # Train the model using the dataset configuration
    model.train(data=f'{root_folder}/dataset.yaml', epochs=epochs, batch=batch_size, imgsz=input_shape, device=[0,1])

    # Save the trained model
    model.save('enhanced_yolov8_vehicle_detection_model.pt')
    
    # Evaluate the model
    val_dataloader = model.dataloader(val=True)  # Load validation data
    evaluate_model(model, val_dataloader)

if __name__ == "__main__":
    main()


In [ ]:
dataset_yaml = """
path: /kaggle/working/juvdv2-vdvwc
train: /kaggle/working/juvdv2-vdvwc/images/Train
val: /kaggle/working/juvdv2-vdvwc/images/Val

nc: 14  # number of classes
names: ['car', 'bike', 'auto', 'rickshaw', 'cycle', 'bus', 'minitruck', 'truck', 'van', 'taxi', 'motorvan', 'toto', 'train', 'boat']
"""
with open('/kaggle/working/juvdv2-vdvwc/dataset.yaml', 'w') as f:
    f.write(dataset_yaml)
